# Introduction to Sarek

Welcome to the interactive introduction to **Sarek**, a high-performance GPGPU framework for OCaml.

In this notebook, we will:
1. Initialize Sarek
2. Define a parallel kernel
3. Execute it using the **Native CPU backend** (OCaml 5 Domains)

In [ ]:
(* Load Sarek *)
#require "sarek";;
#require "sarek.ppx";;

open Sarek;;

## 1. Defining a Kernel

We use the `[%kernel ...]` syntax. This code looks like standard OCaml but is designed to run in parallel.

In [ ]:
let%kernel compute_squares (input : float32 vector) (output : float32 vector) (n : int32) =
  let gid = get_global_id 0 in
  if gid < n then
    output.(gid) <- input.(gid) *. input.(gid)

## 2. Running on the Native Backend

Since Binder environments don't usually have GPUs, we use the `Native Parallel` device which leverages OCaml 5's multicore capabilities.

In [ ]:
(* Select the device *)
let dev = Device.get_device_by_name "Native Parallel" in
Printf.printf "Using: %s
" (Device.name dev);;

(* Prepare data *)
let n = 1000 in
let input = Vector.create Float32 n in
let output = Vector.create Float32 n in
Vector.init Float32 input (fun i -> float_of_int i);;

(* Run the kernel *)
Execute.run compute_squares 
  ~device:dev 
  ~grid:( (n + 255) / 256, 1, 1) 
  ~block:(256, 1, 1) 
  [Vec input; Vec output; Int n];;

(* Check a result *)
Printf.printf "10^2 = %.1f
" (Vector.get output 10);